# pydantic

In [1]:
import pydantic  # pip show pydantic
print(f"our pydantic version: {pydantic.VERSION}")
from pprint import pprint
from typing import List, Optional, Tuple
from pydantic import BaseModel
from pydantic import BaseModel
from pydantic import Field

class Options(BaseModel):
    """單選題的選項物件，包含 A, B, C, D 四個選項"""
    A: str = Field(..., description='選項A')
    B: str = Field(..., description='選項B')
    C: str = Field(..., description='選項C')
    D: str = Field(..., description='選項D')

class MCQ(BaseModel):
    """單選題結構，包含題號、題幹、選項與答案"""
    qid: int = Field(..., description='題號')
    question: str = Field(..., description='題幹')
    options: Options = Field(..., description="本題的四個選項")
    ans: Optional[str] = Field(default=None, description='答案')

class Meta(BaseModel):
    """試題原始資訊，包含 年分、科目、第幾次考試"""
    year: Optional[int] = Field(default=None, description='第?年')
    subject: Optional[str] = Field(default=None, description='科目名稱')
    times: Optional[int] = Field(default=None, description='第?次考試')

class ExtractExam(BaseModel):
    """
    提取整份考卷

    - qset: 單選題考題集合
    - subject: 科目名稱
    - year: 考試年分
    - times: 第幾次考試
    """
    qset: List[MCQ] = Field(..., description='單選題考題')
    metadata: Meta = Field(..., description='考題資訊')

schema = MCQ.model_json_schema()
pprint(schema)

our pydantic version: 2.11.9
{'$defs': {'Options': {'description': '單選題的選項物件，包含 A, B, C, D 四個選項',
                       'properties': {'A': {'description': '選項A',
                                            'title': 'A',
                                            'type': 'string'},
                                      'B': {'description': '選項B',
                                            'title': 'B',
                                            'type': 'string'},
                                      'C': {'description': '選項C',
                                            'title': 'C',
                                            'type': 'string'},
                                      'D': {'description': '選項D',
                                            'title': 'D',
                                            'type': 'string'}},
                       'required': ['A', 'B', 'C', 'D'],
                       'title': 'Options',
                       'type': 'object'}},
 'descript

# load data

In [2]:
from llama_index.readers.file import PDFReader
from pathlib import Path
import time

file_path = Path("./data/114_針灸科學.pdf")
FULL_DOCUMENT=False

pdf_reader = PDFReader(return_full_document=FULL_DOCUMENT)
documents = pdf_reader.load_data(file=file_path)
print(f"len of documents: {len(documents)}")
text = documents[0].text
print(f"text len: {len(text)}")
print('---')
print(text)

len of documents: 14
text len: 871
---
114年第一次專門職業及技術人員高等考試醫師牙醫師中醫師藥師考 
試分階段考試、醫事檢驗師、醫事放射師、物理治療師考試 
代 　 　 號 ： 4 3 1 8 
類 科 名 稱 ： 中 醫 師 ( 二 ) 
科 目 名 稱 ： 中 醫 臨 床 醫 學 （ 四 ） （ 包 括 針 灸 科 學 ） 
考 試 時 間 ： 1 小 時 3 0 分 鐘 座 號 ： _ _ _ _ _ _ _ _ _ _ _ 
  ※注意：本試題禁止使用電子計算器   　　　　　　　  
※本試題為單一選擇題，請選出一個正確或最適當答案。
1 . 常 見 針 灸 配 穴 法 中 ， 所 指 的 「 四 關 穴 」 ， 為 下 列 何 穴 位 之 組 合 ？ 
 
A . 上 星 、 日 月 
B . 合 谷 、 太 衝 
C . 內 關 、 外 關 
D . 上 關 、 下 關 
2 . 依 《 靈 樞 ． 經 脈 》 記 載 ， 「 其 直 者 ， 從 巔 入 絡 腦 ， 還 出 別 下 項 ， 循 肩 膊 內 ， 挾 脊 抵 腰 中 」 ， 指 下 列 
何 經 的 循 行 內 容 ？ 
 
A . 膀 胱 經 
B . 膽 經 
C . 胃 經 
D . 肝 經 
3 . 依 《 靈 樞 ． 經 脈 》 所 記 載 ： 「 是 主 筋 所 生 病 者 ， 痔 、 瘧 、 狂 、 癲 疾 、 頭 顖 、 項 痛 ， 目 黃 、 淚 出 ， 鼽 
衄 ， 項 、 背 、 腰 、 尻 、 膕 、 腨 、 腳 皆 痛 ， 小 趾 不 用 」 ， 指 何 經 的 病 證 內 容 ？ 
 
A . 膀 胱 經 
B . 膽 經 
C . 三 焦 經 
D . 脾 經 
4 . 有 關 腧 穴 之 國 際 譯 名 ， 外 丘 之 編 號 為 何 ？ 
 
A . G B 3 3 
B . G B 3 4 
C . G B 3 5 
D . G B 3 6 
5 . 下 列 何 者 為 陰 維 脈 之 郄 穴 ？ 
 A . 交 信 
B . 築 賓 


# test1: Calling tools directly

In [4]:
from llama_index.core.program.function_program import get_function_tool

exam_tool = get_function_tool(ExtractExam)
print(f"# tool info: ")
print(f"# name: {exam_tool.metadata.name}\n\n# description: {exam_tool.metadata.description}")
print('---')

# pip install llama-index-llms-ollama
from llama_index.llms.ollama import Ollama
llama = Ollama(
    model="llama3.1:latest",
    request_timeout=120.0,
    context_window=8000,
    temperature=0.0,
)

start = time.time()
resp = llama.chat_with_tools(
    [exam_tool],
    user_msg="請從下列文本中提取考試: " + text,
    tool_required=True,  # can optionally force the tool call
)
end = time.time()
print(f'dur: {end-start:.2f} sec')
tool_calls = llama.get_tool_calls_from_response(
    resp, error_on_no_tool_call=False
)
print(f"type: {type(tool_calls)}, len: {len(tool_calls)}, dtype: {type(tool_calls[0])}")
print('---')
pprint(tool_calls[0].tool_kwargs)

# tool info: 
# name: ExtractExam

# description: 提取整份考卷

- qset: 單選題考題集合
- subject: 科目名稱
- year: 考試年分
- times: 第幾次考試
---


2025-10-01 17:15:34,959 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


dur: 13.84 sec
type: <class 'list'>, len: 1, dtype: <class 'llama_index.core.llms.llm.ToolSelection'>
---
{'metadata': {'subject': '中醫師(二)', 'times': 1, 'year': 114},
 'qset': [{'ans': None,
           'options': {'A': '上星、日月', 'B': '合谷、太衝', 'C': '內關、外關', 'D': '上關、下關'},
           'qid': 1,
           'question': '常見針灸配穴法中，所指的「四關穴」，為下列何穴位之組合？'},
          {'ans': None,
           'options': {'A': '膀胱經', 'B': '膽經', 'C': '胃經', 'D': '肝經'},
           'qid': 2,
           'question': '依《靈樞．經脈》記載，「其直者，從巔入絡腦，還出別下項，循肩膊內，挾脊抵腰中」，指下列何經的循行內容？'},
          {'ans': None,
           'options': {'A': '膀胱經', 'B': '膽經', 'C': '三焦經', 'D': '脾經'},
           'qid': 3,
           'question': '依《靈樞．經脈》所記載：「是主筋所生病者，痔、瘧、狂、癲疾、頭顖、項痛，目黃、淚出，鼻衄，項、背、腰、尻、膕、腨、腳皆痛，小趾不用」，指何經的病證內容？'},
          {'ans': None,
           'options': {'A': 'G B 33',
                       'B': 'G B 34',
                       'C': 'G B 35',
                       'D': 'G B 36'},
           'qid': 4,
           'question': '有關俞穴之國際譯名，外丘之編號為何？

# test2: allow multiple tool calls

In [5]:
mcq_tool = get_function_tool(MCQ)
print(f"# name: {mcq_tool.metadata.name}\n\n# description: {mcq_tool.metadata.description}")

start = time.time()
resp = llama.chat_with_tools(
    [mcq_tool],
    user_msg="你是一個無情的考題提取機器，負責從文本中盡可能多的提取 MCQ，以下是文本資訊：" + text,
    tool_required=True,  # can optionally force the tool call
    allow_parallel_tool_calls=True,
)
end = time.time()
print(f"dur: {end - start:.2f} sec")
tool_calls = llama.get_tool_calls_from_response(
    resp, error_on_no_tool_call=False
)
print(f'len of tool_call: {len(tool_calls)}')
print('---')
for tool_call in tool_calls:
    pprint(tool_call.tool_kwargs)

# name: MCQ

# description: 單選題結構，包含題號、題幹、選項與答案


2025-10-01 17:15:44,320 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


dur: 7.18 sec
len of tool_call: 1
---
{'ans': 'C',
 'options': {'A': '前 衡 , 南 千',
             'B': '手机 , 卯 双',
             'C': '全 黑 , 割 黑',
             'D': '丽 黑 , 不 黑'},
 'qid': 1,
 'question': '台为 屭等 手机 先家 有的 分果 面工 、 , 我的 台为家等手机先家一些分果面工 , 。'}


# test3: gpt-5-mini with multiple tool calls

In [6]:
print(f"# name: {mcq_tool.metadata.name}\n\n# description: {mcq_tool.metadata.description}")
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())
from llama_index.llms.openai import OpenAI
mini = OpenAI(model="gpt-5-mini")

start = time.time()
resp = mini.chat_with_tools(
    [mcq_tool],
    user_msg="你是一個無情的考題提取機器，負責從文本中盡可能多的提取 MCQ，以下是文本資訊：" + text,
    tool_required=True,  # can optionally force the tool call
    allow_parallel_tool_calls=True,
)
end = time.time()
print(f"dur: {end - start:.2f} sec")
tool_calls = mini.get_tool_calls_from_response(
    resp, error_on_no_tool_call=False
)
print(f'len of tool_call: {len(tool_calls)}')
print('---')
for tool_call in tool_calls:
    pprint(tool_call.tool_kwargs)

# name: MCQ

# description: 單選題結構，包含題號、題幹、選項與答案


2025-10-01 17:21:17,820 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


dur: 36.84 sec
len of tool_call: 5
---
{'ans': None,
 'options': {'A': '上星、日月', 'B': '合谷、太衝', 'C': '內關、外關', 'D': '上關、下關'},
 'qid': 1,
 'question': '常見針灸配穴法中，所指的「四關穴」，為下列何穴位之組合？'}
{'ans': None,
 'options': {'A': '膀胱經', 'B': '膽經', 'C': '胃經', 'D': '肝經'},
 'qid': 2,
 'question': '依《靈樞．經脈》記載，「其直者，從巔入絡腦，還出別下項，循肩膊內，挾脊抵腰中」，指下列何經的循行內容？'}
{'ans': None,
 'options': {'A': '膀胱經', 'B': '膽經', 'C': '三焦經', 'D': '脾經'},
 'qid': 3,
 'question': '依《靈樞．經脈》所記載：「是主筋所生病者，痔、瘧、狂、癲疾、頭顱、項痛，目黃、淚出，鼻衄，項、背、腰、尻、膕、腨、腳皆痛，小趾不用」，指何經的病證內容？'}
{'ans': None,
 'options': {'A': 'GB33', 'B': 'GB34', 'C': 'GB35', 'D': 'GB36'},
 'qid': 4,
 'question': '有關腧穴之國際譯名，外丘之編號為何？'}
{'ans': None,
 'options': {'A': '交信', 'B': '築賓', 'C': '', 'D': ''},
 'qid': 5,
 'question': '下列何者為陰維脈之郄穴？'}


# test4: gemma without json mode

In [9]:
import json
schema = MCQ.model_json_schema()
prompt = "Here is a JSON schema for an Exam: " + json.dumps(
    schema, indent=2, ensure_ascii=False
)

gemma = Ollama(
    model="gemma3:12b",
    request_timeout=120.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
    json_mode=False,
    temperature=0.0,
)

prompt += (
    """
  Extract an Exam from the following text.
  Format your output as a JSON object according to the schema above.
  Do not include any other text than the JSON object.
  Omit any markdown formatting. Do not include any preamble or explanation.
  請盡可能多的提取考題
"""
    + text
)

response = gemma.complete(prompt)



2025-10-01 17:32:06,955 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


```json
[
  {
    "qid": 1,
    "question": "常見針灸配穴法中，所指的「四關穴」，為下列何穴位之組合？",
    "options": {
      "A": "上星、日月",
      "B": "合谷、太衝",
      "C": "內關、外關",
      "D": "上關、下關"
    },
    "ans": null
  },
  {
    "qid": 2,
    "question": "依《靈樞．經脈》記載：「其直者，從巔入絡腦，還出別下項，循肩膊內，挾脊抵腰中」，指下列何經的循行內容？",
    "options": {
      "A": "膀胱經",
      "B": "膽經",
      "C": "胃經",
      "D": "肝經"
    },
    "ans": null
  },
  {
    "qid": 3,
    "question": "依《靈樞．經脈》所記載：「是主筋所生病者，痔、瘧、狂、癲疾、頭顖、項痛，目黃、淚出，鼽衄，項、背、腰、尻、膕、腨、腳皆痛，小趾不用」，指何經的病證內容？",
    "options": {
      "A": "膀胱經",
      "B": "膽經",
      "C": "三焦經",
      "D": "脾經"
    },
    "ans": null
  },
  {
    "qid": 4,
    "question": "有關腧穴之國際譯名，外丘之編號為何？",
    "options": {
      "A": "G B 3 3",
      "B": "G B 3 4",
      "C": "G B 3 5",
      "D": "G B 3 6"
    },
    "ans": null
  },
  {
    "qid": 5,
    "question": "下列何者為陰維脈之郄穴？",
    "options": {
      "A": "交信",
      "B": "築賓"
    },
    "ans": null
  }
]
```


In [14]:
import re

raw = response.text.strip()

# 把 ```json ... ``` 和 ``` 拿掉
if raw.startswith("```"):
    raw = re.sub(r"^```(?:json)?", "", raw)
    raw = re.sub(r"```$", "", raw)
    raw = raw.strip()

data = json.loads(raw)
pprint(data)


[{'ans': None,
  'options': {'A': '上星、日月', 'B': '合谷、太衝', 'C': '內關、外關', 'D': '上關、下關'},
  'qid': 1,
  'question': '常見針灸配穴法中，所指的「四關穴」，為下列何穴位之組合？'},
 {'ans': None,
  'options': {'A': '膀胱經', 'B': '膽經', 'C': '胃經', 'D': '肝經'},
  'qid': 2,
  'question': '依《靈樞．經脈》記載：「其直者，從巔入絡腦，還出別下項，循肩膊內，挾脊抵腰中」，指下列何經的循行內容？'},
 {'ans': None,
  'options': {'A': '膀胱經', 'B': '膽經', 'C': '三焦經', 'D': '脾經'},
  'qid': 3,
  'question': '依《靈樞．經脈》所記載：「是主筋所生病者，痔、瘧、狂、癲疾、頭顖、項痛，目黃、淚出，鼽衄，項、背、腰、尻、膕、腨、腳皆痛，小趾不用」，指何經的病證內容？'},
 {'ans': None,
  'options': {'A': 'G B 3 3', 'B': 'G B 3 4', 'C': 'G B 3 5', 'D': 'G B 3 6'},
  'qid': 4,
  'question': '有關腧穴之國際譯名，外丘之編號為何？'},
 {'ans': None,
  'options': {'A': '交信', 'B': '築賓'},
  'qid': 5,
  'question': '下列何者為陰維脈之郄穴？'}]


# test6: gemma with json mode

In [17]:
json_gemma = Ollama(
    model="gemma3:12b",
    request_timeout=120.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
    json_mode=True,
    temperature=0.0,
)
response = json_gemma.complete(prompt)
json.loads(response.text)

2025-10-01 17:38:57,749 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


{'qid': 1,
 'question': '常見針灸配穴法中，所指的是「四關穴」，為下列何穴位之組合？',
 'options': {'A': '上星、日月', 'B': '合谷、太衝', 'C': '內關、外關', 'D': '上關、下關'},
 'ans': None}

# test7: exam schema

In [18]:
schema = ExtractExam.model_json_schema()
prompt = "Here is a JSON schema for an Exam: " + json.dumps(
    schema, indent=2, ensure_ascii=False
)

json_gemma = Ollama(
    model="gemma3:12b",
    request_timeout=120.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
    json_mode=True,
    temperature=0.0,
)


prompt += (
    """
  Extract an Exam from the following text.
  Format your output as a JSON object according to the schema above.
  Do not include any other text than the JSON object.
  Omit any markdown formatting. Do not include any preamble or explanation.
  請盡可能多的提取考題
"""
    + text
)

response = json_gemma.complete(prompt)
json.loads(response.text)

2025-10-01 17:42:07,973 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


{'qset': [{'qid': 1,
   'question': '常見針灸配穴法中，所指的「四關穴」，為下列何穴位之組合？',
   'options': {'A': '上星、日月', 'B': '合谷、太衝', 'C': '內關、外關', 'D': '上關、下關'}},
  {'qid': 2,
   'question': '依《靈樞．經脈》記載：「其直者，從巔入絡腦，還出別下項，循肩膊內，挾脊抵腰中」，指下列何經的循行內容？',
   'options': {'A': '膀胱經', 'B': '膽經', 'C': '胃經', 'D': '肝經'}},
  {'qid': 3,
   'question': '依《靈樞．經脈》所記載：「是主筋所生病者，痔、瘧、狂、癲疾、頭眩、項痛，目黃、淚出，鼽衄，項、背、腰、尻、膕、腨、腳皆痛，小趾不用」，指何經的病證內容？',
   'options': {'A': '膀胱經', 'B': '膽經', 'C': '三焦經', 'D': '脾經'}},
  {'qid': 4,
   'question': '有關腧穴之國際譯名，外丘之編號為何？',
   'options': {'A': 'G B 3 3',
    'B': 'G B 3 4',
    'C': 'G B 3 5',
    'D': 'G B 3 6'}},
  {'qid': 5, 'question': '下列何者為陰維脈之郄穴？', 'options': {'A': '交信', 'B': '築賓'}}],
 'metadata': {'year': 114, 'subject': '中醫師 ( 二 )', 'times': 1}}